In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/train.csv", index_col=[0])

In [3]:
r_masks = [
    ["zoning", df["MSZoning"] == "RM"],
    ["lotconfig", df["LotConfig"].isin(["Inside", "FR2"])],
    ["lotarea", (df["LotArea"] > 8000) & (df["LotArea"] < 12_000)]
]

In [88]:
masks = [mask for _, mask in r_masks]

combined_mask = np.column_stack(masks)

abs_removed = - ((np.sum(combined_mask, axis=0) - np.size(df, axis=0)) / np.size(df, axis=0) * 100)

In [89]:
filtered_counts = {
    name: {
        "absolute": absolute,
        "relative": 0,
    } 
    for name, absolute in zip([name for name, _ in r_masks], - abs_removed)
}

In [102]:
counter = 1
previous_feature = None

for name, _ in r_masks:
    cum_mask = np.logical_and.reduce(combined_mask[:,:counter], axis=1).sum()
    
    filtered_counts[name]["relative"] = (cum_mask - df.shape[0]) / df.shape[0] * 100 if not previous_feature else (cum_mask - previous_remaining) / previous_remaining * 100
    
    previous_remaining = np.logical_and.reduce(combined_mask[:,:counter], axis=1).sum()
    
    previous_feature = name 
    counter += 1

In [103]:
pd.DataFrame(filtered_counts).T.round(2)

,absolute,relative
zoning,-85.07,-85.07
lotconfig,-24.73,-18.35
lotarea,-51.44,-78.65
